In [1]:
!pip3 install datasets
!pip3 install transformers 
!pip3 install seqeval
!pip3 install transformers -U
!pip3 install transformers[torch]

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
zsh:1: no matches found: transformers[torch]


In [2]:
import transformers
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer

from datasets import load_dataset,concatenate_datasets, load_metric
import pandas as pd
import numpy as np

In [3]:
wnut = load_dataset("wnut_17")

Found cached dataset wnut_17 (/Users/evan/.cache/huggingface/datasets/wnut_17/wnut_17/1.0.0/077c7f08b8dbc800692e8c9186cdf3606d5849ab0e7be662e6135bb10eba54f9)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
label_list = wnut["train"].features[f"ner_tags"].feature.names
id2tag = {id: tag for id, tag in enumerate(label_list)}
id2tag

{0: 'O',
 1: 'B-corporation',
 2: 'I-corporation',
 3: 'B-creative-work',
 4: 'I-creative-work',
 5: 'B-group',
 6: 'I-group',
 7: 'B-location',
 8: 'I-location',
 9: 'B-person',
 10: 'I-person',
 11: 'B-product',
 12: 'I-product'}

In [5]:
# merge train & validation sets
from datasets import concatenate_datasets

train_dataset = concatenate_datasets([wnut["train"],wnut["validation"]])
train_dataset

Dataset({
    features: ['id', 'tokens', 'ner_tags'],
    num_rows: 4403
})

In [7]:
ith_example=2

print(train_dataset[ith_example]['tokens'])
print(train_dataset[ith_example]['ner_tags'])
print([id2tag[label] for label in train_dataset[ith_example]['ner_tags']])

['Pxleyes', 'Top', '50', 'Photography', 'Contest', 'Pictures', 'of', 'August', '2010', '...', 'http://bit.ly/bgCyZ0', '#photography']
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['B-corporation', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [8]:
model_name = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [9]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [10]:
tokenized_wnut = wnut.map(tokenize_and_align_labels, batched=True)
tokenized_train_dataset = train_dataset.map(tokenize_and_align_labels, batched=True)
tokenized_train_dataset

Loading cached processed dataset at /Users/evan/.cache/huggingface/datasets/wnut_17/wnut_17/1.0.0/077c7f08b8dbc800692e8c9186cdf3606d5849ab0e7be662e6135bb10eba54f9/cache-2fe5dae7720f7a18.arrow
Loading cached processed dataset at /Users/evan/.cache/huggingface/datasets/wnut_17/wnut_17/1.0.0/077c7f08b8dbc800692e8c9186cdf3606d5849ab0e7be662e6135bb10eba54f9/cache-3c176d959584da3a.arrow
Loading cached processed dataset at /Users/evan/.cache/huggingface/datasets/wnut_17/wnut_17/1.0.0/077c7f08b8dbc800692e8c9186cdf3606d5849ab0e7be662e6135bb10eba54f9/cache-3e12ab8b5661768d.arrow


Map:   0%|          | 0/4403 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 4403
})

In [11]:
import pandas as pd

# Convert the dataset to a Pandas DataFrame
train_df = pd.DataFrame(tokenized_train_dataset.to_pandas())

train_df

,id,tokens,ner_tags,input_ids,token_type_ids,attention_mask,labels
0,0,"[@paulwalk, It, 's, the, view, from, where, I,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, ...","[101, 1030, 2703, 17122, 2009, 1005, 1055, 199...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, -100, -100, 0, 0, -100, 0, 0, 0, 0, ..."
1,1,"[From, Green, Newsfeed, :, AHFA, extends, dead...","[0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[101, 2013, 2665, 2739, 7959, 2098, 1024, 6289...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, -100, -100, 0, 5, -100, 0, 0, ..."
2,2,"[Pxleyes, Top, 50, Photography, Contest, Pictu...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[101, 1052, 20959, 2229, 2327, 2753, 5855, 504...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 1, -100, -100, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,3,"[today, is, my, last, day, at, the, office, .]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[101, 2651, 2003, 2026, 2197, 2154, 2012, 1996...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]"
4,4,"[4Dbling, 's, place, til, monday, ,, party, pa...","[9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[101, 1018, 18939, 2989, 1005, 1055, 2173, 186...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 9, -100, -100, 0, -100, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...
4398,1004,"[your, sarcasm, is, goals, :, D, :, D]","[0, 0, 0, 0, 0, 0, 0, 0]","[101, 2115, 20954, 2003, 3289, 1024, 1040, 102...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[-100, 0, 0, 0, 0, 0, 0, 0, 0, -100]"
4399,1005,"[I, HATE, THIS, I, HATE, THIS, I, HATE, THIS, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[101, 1045, 5223, 2023, 1045, 5223, 2023, 1045...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100, 0, ..."
4400,1006,"[For, education, ., Lol, yeah, like, my, dads,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[101, 2005, 2495, 1012, 8840, 2140, 3398, 2066...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, -100, 0, 0, 0, 0, -100, 0, ..."
4401,1007,"[excellent, poem, Dubbz﻿]","[0, 0, 9]","[101, 6581, 5961, 12931, 2497, 2480, 102]","[0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1]","[-100, 0, 0, 9, -100, -100, -100]"


In [45]:
def compute_reward(pred_labels, true_labels):
    num_O = sum([1 for i in true_labels if i == 'O'])
    num_entity = len(true_labels) - num_O

    num_O_correct = sum([1 for (p, l) in zip(pred_labels, true_labels) if p == l and l == 'O'])
    num_entity_correct = sum([1 for (p, l) in zip(pred_labels, true_labels) if p == l and l != 'O'])
    
    O_rate = num_O_correct/num_O if num_O != 0 else 0
    entity_rate = num_entity_correct/num_entity if num_entity != 0 else 0

    if num_O == 0:
        reward = entity_rate * 10
        print(2)
        return reward
    if num_entity == 0:
        reward = O_rate * 10
        print(1)
        return reward
        
    reward = O_rate * 4 + entity_rate * 6
    return reward

In [47]:
pred_labels = ['I-location', 'O', 'O', 'O', 'O', 'I-group', 'O', 'I-product', 'I-product', 'O', 'O', 'O', 'O', 'O', 'O', 'B-location', 'O', 'O', 'I-group', 'O', 'O', 'O', 'O', 'O', 'O', 'I-group', 'I-product', 'I-product', 'I-location', 'O', 'I-location', 'O', 'O', 'B-person']

true_labels = ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-location', 'I-location', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


compute_reward(pred_labels, true_labels)

2.5

In [1]:
import pandas as pd

# read in the four dataframes
df1 = pd.read_csv("train_df_avg_reward_0.3571.csv")
df2 = pd.read_csv("train_df_avg_reward_4.0345.csv")
df3 = pd.read_csv("train_df_avg_reward_7.4654.csv")
df4 = pd.read_csv("train_df_avg_reward_10.0.csv")

# concatenate the dataframes
concatenated_df = pd.concat([df1, df2, df3, df4])

# shuffle the concatenated dataframe
concatenated_df = concatenated_df.sample(frac=1, random_state=42).reset_index(drop=True)

# view the concatenated dataframe
print(concatenated_df)

         id                                             tokens   
0      3014  ['Fourteen' 'chapters' 'in' '.' 'January' "'s"...  \
1       103  ['We' 'are' 'one' 'step' 'closer' 'to' 'our' '...   
2       210  ['Man' ':' 'Hey' 'there' ',' 'havent' 'I' 'see...   
3      1911  ['Gonna' 'be' 'on' 'national' 'television' 'mo...   
4      2513  ['RT' '@followFDD' ':' 'The' 'case' 'for' 'dea...   
...     ...                                                ...   
17607  2478  ['literally' 'though' 'Saturday' 'meeting' 'th...   
17608  3158  ['#NowPlaying' ':' 'I' 'LOVE' 'MAKONNEN' 'Feat...   
17609   987  ['RT' '@Desbishop' ':' 'Just' 'about' 'to' 'go...   
17610   860  ['Hurry' 'up' '!' 'Brandy' 'will' 'be' 'Leavin...   
17611  2586  ['Also' 'having' 'a' 'hard' 'time' 'figuring' ...   

                                                ner_tags   
0                        [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]  \
1            [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]   
2      [0 0 0 0 0 0 0 0 0 0

In [2]:
concatenated_df.to_csv("training_data.csv", index=False)